In [1]:
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

In [2]:
# conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities")
# sc = SparkContext(conf = conf)

print "\nLoading movie names..."


Loading movie names...


In [40]:
def loadMovieNames():
    movieNames = {}
    with open("/home/cloudera/anurag/itemfile.txt") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1].decode('ascii', 'ignore')
    return movieNames
nameDict = loadMovieNames()

In [4]:
nameDict

{1: u'Toy Story (1995)',
 2: u'GoldenEye (1995)',
 3: u'Four Rooms (1995)',
 4: u'Get Shorty (1995)',
 5: u'Copycat (1995)',
 6: u'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 7: u'Twelve Monkeys (1995)',
 8: u'Babe (1995)',
 9: u'Dead Man Walking (1995)',
 10: u'Richard III (1995)',
 11: u'Seven (Se7en) (1995)',
 12: u'Usual Suspects, The (1995)',
 13: u'Mighty Aphrodite (1995)',
 14: u'Postino, Il (1994)',
 15: u"Mr. Holland's Opus (1995)",
 16: u'French Twist (Gazon maudit) (1995)',
 17: u'From Dusk Till Dawn (1996)',
 18: u'White Balloon, The (1995)',
 19: u"Antonia's Line (1995)",
 20: u'Angels and Insects (1995)',
 21: u'Muppet Treasure Island (1996)',
 22: u'Braveheart (1995)',
 23: u'Taxi Driver (1976)',
 24: u'Rumble in the Bronx (1995)',
 25: u'Birdcage, The (1996)',
 26: u'Brothers McMullen, The (1995)',
 27: u'Bad Boys (1995)',
 28: u'Apollo 13 (1995)',
 29: u'Batman Forever (1995)',
 30: u'Belle de jour (1967)',
 31: u'Crimson Tide (1995)',
 32: u'Crumb (1994)',

In [41]:
nameDict[1]

u'Toy Story (1995)'

In [1]:
data = sc.textFile("file:///home/cloudera/anurag/datafile.txt")


In [2]:
# Map ratings to key / value pairs: user ID => movie ID, rating

mapped  =  data.map(lambda l:l.split()).map(lambda l:(int(l[0]),(int(l[1]),float(l[2]))))

In [3]:
mapped.take(4)

[(196, (242, 3.0)), (186, (302, 3.0)), (22, (377, 1.0)), (244, (51, 2.0))]

In [4]:
# Emit every movie rated together by the same user.
# Self-join to find every combination.
joinedRatings = mapped.join(mapped)

In [5]:
joinedRatings.take(4)

[(2, ((292, 4.0), (292, 4.0))),
 (2, ((292, 4.0), (251, 5.0))),
 (2, ((292, 4.0), (50, 5.0))),
 (2, ((292, 4.0), (314, 1.0)))]

In [6]:
#remove duplicate movie ratings
def removeDuplicates((userID,Ratings)):
    (movie1,Rating1) = Ratings[0]
    (movie2,Rating2) = Ratings[1]
    return movie1<movie2

UniqueJoinedRatings = joinedRatings.filter(removeDuplicates)
    

In [7]:
UniqueJoinedRatings.take(4)

[(2, ((292, 4.0), (314, 1.0))),
 (2, ((292, 4.0), (297, 4.0))),
 (2, ((292, 4.0), (312, 3.0))),
 (2, ((292, 4.0), (303, 4.0)))]

In [8]:
def makePairs((user, ratings)):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))
# Now key by (movie1, movie2) pairs.
moviePairs = UniqueJoinedRatings.map(makePairs)

In [9]:
moviePairs.take(4)

[((292, 314), (4.0, 1.0)),
 ((292, 297), (4.0, 4.0)),
 ((292, 312), (4.0, 3.0)),
 ((292, 303), (4.0, 4.0))]

In [35]:
moviePairRatings = moviePairs.groupByKey()
moviePairRatings.take(1)

[((197, 1097), <pyspark.resultiterable.ResultIterable at 0x7f1681f28190>)]

In [36]:
moviePairRatingsnew = moviePairs.groupByKey().mapValues(list)

In [37]:
moviePairRatingsnew.take(4)

[((129, 201),
  [(4.0, 3.0),
   (4.0, 5.0),
   (4.0, 3.0),
   (2.0, 3.0),
   (5.0, 5.0),
   (4.0, 2.0),
   (4.0, 4.0),
   (5.0, 5.0),
   (3.0, 4.0),
   (3.0, 3.0),
   (5.0, 3.0),
   (4.0, 3.0),
   (5.0, 3.0),
   (4.0, 5.0),
   (5.0, 4.0),
   (4.0, 4.0),
   (1.0, 5.0),
   (4.0, 4.0),
   (5.0, 5.0),
   (3.0, 3.0),
   (4.0, 5.0),
   (3.0, 4.0),
   (5.0, 5.0),
   (4.0, 3.0),
   (2.0, 3.0),
   (3.0, 3.0),
   (3.0, 4.0),
   (5.0, 4.0)]),
 ((384, 1530), [(2.0, 2.0), (3.0, 1.0)]),
 ((678, 1276), [(4.0, 4.0), (2.0, 1.0), (3.0, 3.0), (2.0, 1.0), (2.0, 3.0)]),
 ((500, 1322), [(2.0, 2.0)])]

In [53]:
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

In [54]:
movieID = 129

In [55]:
scoreThreshold = 0.10
coOccurenceThreshold = 2

In [56]:
# Filter for movies with this sim that are "good" as defined by
# our quality thresholds above
filteredResults = moviePairSimilarities.filter(lambda((pair,sim)):(pair[0] == movieID or pair[1] == movieID) and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)


In [1]:
moviePairSimilarities.sortByKey()

In [2]:
results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)


In [ ]:
results